In [1]:
#!pip install opencv-python

In [2]:
#!pip list

In [8]:
#!pip install mediapipe --user

  Using cached mediapipe-0.9.0.1-cp37-cp37m-win_amd64.whl (49.8 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached flatbuffers-23.3.3-py2.py3-none-any.whl (26 kB)
  Using cached opencv_contrib_python-4.7.0.72-cp37-abi3-win_amd64.whl (44.9 MB)


You should consider upgrading via the 'C:\Users\cscs7\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import mediapipe as mp
#from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import time
import stat
import os
import csv
import re
from math import acos, degrees
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist

In [2]:
def dp(dist_mat):
    """
    Find minimum-cost path through matrix `dist_mat` using dynamic programming.

    The cost of a path is defined as the sum of the matrix entries on that
    path. See the following for details of the algorithm:

    - http://en.wikipedia.org/wiki/Dynamic_time_warping
    - https://www.ee.columbia.edu/~dpwe/resources/matlab/dtw/dp.m

    The notation in the first reference was followed, while Dan Ellis's code
    (second reference) was used to check for correctness. Returns a list of
    path indices and the cost matrix.
    """

    N, M = dist_mat.shape
    
    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)

In [3]:
def calcular_costos(ej_usuario, df_planchas_experto1, inicio, fin):
    
    inicio=0
    fin = 1
    
    resultados = []
    resultados_index = []
    resultados_costo_al = []
    resultados_costo_al_normalizado = []

    #while (inicio <= len(df_planchas_experto1)-1 and fin <= len(df_planchas_experto1)):
        #print()
    ej_experto = []
    
    for i in df_planchas_experto1.columns:
            #print(i)
        ej_experto.append(df_planchas_experto1[i][inicio])

    x = np.array(ej_usuario) 

    y = np.array(ej_experto)

        #plt.figure(figsize=(6, 4))
        #plt.plot(np.arange(x.shape[0]), x + 1.5, "-o", c="C3")
        #plt.plot(np.arange(y.shape[0]), y - 1.5, "-o", c="C0")
        #plt.axis("off")
        #plt.savefig("signals_a_b.pdf")

        # Distance matrix
    N = x.shape[0]
    M = y.shape[0]
    dist_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            dist_mat[i, j] = abs(x[i] - y[j])

    # DTW
    path, cost_mat = dp(dist_mat)
        #print("Alignment cost: {:.4f}".format(cost_mat[N - 1, M - 1]))
        #print("Normalized alignment cost: {:.4f}".format(cost_mat[N - 1, M - 1]/(N + M)))

        #plt.figure(figsize=(6, 4))
        #plt.subplot(121)
        #plt.title("Distance matrix")
        #plt.imshow(dist_mat, cmap=plt.cm.binary, interpolation="nearest", origin="lower")
        #plt.subplot(122)
        #plt.title("Cost matrix")
        #plt.imshow(cost_mat, cmap=plt.cm.binary, interpolation="nearest", origin="lower")
    x_path, y_path = zip(*path)
        #plt.plot(y_path, x_path);
        #[indice, aligment_cost, normalized_alignment_cost:]
    resultados_index.append(inicio)
    resultados_costo_al.append(cost_mat[N - 1, M - 1])
    resultados_costo_al_normalizado.append(cost_mat[N - 1, M - 1]/(N + M))
    resultados.append([inicio,cost_mat[N - 1, M - 1],cost_mat[N - 1, M - 1]/(N + M)])
    inicio=inicio+1
    fin = fin+1
    return resultados

In [4]:
# invocamos a la solucion para graficar puntos

mp_drawing = mp.solutions.drawing_utils

In [5]:
#invocamos a la solución pose

mp_pose = mp.solutions.pose

## Para imágenes

### Extracción de puntos de las imágenes

In [167]:
imagenes_expertos = ["front_plank1_1.png","front_plank1_2.png","front_plank1_3.png",#"lunges1_4.png","lunges1_5.png",
                    "front_plank3_1.png","front_plank3_2.png","front_plank3_3.png",#"lunges2_4.png","lunges2_5.png",
                    "front_plank4_1.png","front_plank4_2.png","front_plank4_3.png"]#"lunges3_4.png","lunges3_5.png"]

#right_torso_angles = [165, 175, 165]
#right_arm_angles = [155, 70, 155]
#right_leg_angles = [170, 160, 170]
c = 0
experto = 1

df_puntos_final = pd.DataFrame()

for i in imagenes_expertos:

    with mp_pose.Pose(static_image_mode=True) as pose:
        ruta = "./nuevas imagenes/front_plank/"+str(i)
        image = cv2.imread(ruta,1)
        #cv2.imshow("Image", image)

        height, width, _ = image.shape
      #transformamos la imagen de entrada a rgb, ya que así las lee open cv
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        print("Pose landmarks: ", results.pose_landmarks)

      #dibujamos los puntos de referencia y sus conexiones

        if results.pose_landmarks is not None:

            resultados = []
            
            for i in range(0, len(results.pose_landmarks.landmark)):

                resultados.append(results.pose_landmarks.landmark[i].x)
                resultados.append(results.pose_landmarks.landmark[i].y)
                resultados.append(results.pose_landmarks.landmark[i].z)
                resultados.append(results.pose_landmarks.landmark[i].visibility)
                
            landmarks = results.pose_landmarks.landmark
            
            right_arm_x1 = int(landmarks[13].x * width) #right_arm
            right_arm_x2 = int(landmarks[11].x * width)
            right_arm_x3 = int(landmarks[23].x * width)
            right_arm_y1 = int(landmarks[13].y * height)
            right_arm_y2 = int(landmarks[11].y * height)
            right_arm_y3 = int(landmarks[23].y * height)  

            right_arm_p1 = np.array([right_arm_x1, right_arm_y1])
            right_arm_p2 = np.array([right_arm_x2, right_arm_y2])
            right_arm_p3 = np.array([right_arm_x3, right_arm_y3])

            right_arm_l1 = np.linalg.norm(right_arm_p2 - right_arm_p3)
            right_arm_l2 = np.linalg.norm(right_arm_p1 - right_arm_p3)
            right_arm_l3 = np.linalg.norm(right_arm_p1 - right_arm_p2)

            right_arm_angle=degrees(acos((right_arm_l1**2+right_arm_l3**2-right_arm_l2**2)/(2*right_arm_l1*right_arm_l3)))

            right_torso_x1 = int(landmarks[11].x * width) #right_torso
            right_torso_x2 = int(landmarks[23].x * width)
            right_torso_x3 = int(landmarks[25].x * width) 
            right_torso_y1 = int(landmarks[11].y * height)
            right_torso_y2 = int(landmarks[23].y * height)
            right_torso_y3 = int(landmarks[25].y * height) 

            right_torso_p1 = np.array([right_torso_x1, right_torso_y1])
            right_torso_p2 = np.array([right_torso_x2, right_torso_y2])
            right_torso_p3 = np.array([right_torso_x3, right_torso_y3])

            right_torso_l1 = np.linalg.norm(right_torso_p2 - right_torso_p3)
            right_torso_l2 = np.linalg.norm(right_torso_p1 - right_torso_p3)
            right_torso_l3 = np.linalg.norm(right_torso_p1 - right_torso_p2)

            right_torso_angle=degrees(acos((right_torso_l1**2+right_torso_l3**2-right_torso_l2**2)/(2*right_torso_l1*right_torso_l3)))

            right_leg_x1 = int(landmarks[25].x * width) #right_leg
            right_leg_x2 = int(landmarks[27].x * width)
            right_leg_x3 = int(landmarks[31].x * width) 
            right_leg_y1 = int(landmarks[25].y * height)
            right_leg_y2 = int(landmarks[27].y * height)
            right_leg_y3 = int(landmarks[31].y * height)
            
            right_leg_p1 = np.array([right_leg_x1, right_leg_y1])
            right_leg_p2 = np.array([right_leg_x2, right_leg_y2])
            right_leg_p3 = np.array([right_leg_x3, right_leg_y3])

            right_leg_l1 = np.linalg.norm(right_leg_p2 - right_leg_p3)
            right_leg_l2 = np.linalg.norm(right_leg_p1 - right_leg_p3)
            right_leg_l3 = np.linalg.norm(right_leg_p1 - right_leg_p2)

            right_leg_angle=degrees(acos((right_leg_l1**2+right_leg_l3**2-right_leg_l2**2)/(2*right_leg_l1*right_leg_l3)))
                
            df_puntos = pd.DataFrame(np.reshape(resultados, (132, 1)).T)
            df_puntos['pose'] = c+1
            df_puntos['right_shoulder_angles'] =  right_arm_angle#right_arm_angles[c]
            df_puntos['right_hip_angles'] = right_torso_angle #right_torso_angles[c]
            df_puntos['right_ankle_angles'] = right_leg_angle #right_leg_angles[c]
            #df_puntos['desv'] = 5
            df_puntos_final = pd.concat([df_puntos_final, df_puntos])
            
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(128,0,250), thickness=2, circle_radius=3),
                                  mp_drawing.DrawingSpec(color=(255,255,255), thickness=2))

        cv2.imshow("Image", image)
        cv2.waitKey(0)
        
        c = c + 1
        
        if c % 3 == 0:
            
            df_puntos_final.columns = ["nose_x","nose_y","nose_z","nose_visibility",
                                        "left_eye_inner_x","left_eye_inner_y","left_eye_inner_z","left_eye_inner_visibility",
                                        "left_eye_x","left_eye_y","left_eye_z","left_eye_visibility",
                                        "left_eye_outer_x","left_eye_outer_y","left_eye_outer_z","left_eye_outer_visibility",
                                        "right_eye_inner_x","right_eye_inner_y","right_eye_inner_z","right_eye_inner_visibility",
                                        "right_eye_x","right_eye_y","right_eye_z","right_eye_visibility",
                                        "right_eye_outer_x","right_eye_outer_y","right_eye_outer_z","right_eye_outer_visibility",
                                        "left_ear_x","left_ear_y","left_ear_z","left_ear_visibility",
                                        "right_ear_x","right_ear_y","right_ear_z","right_ear_visibility",
                                        "mouth_left_x","mouth_left_y","mouth_left_z","mouth_left_visibility",
                                        "mouth_right_x","mouth_right_y","mouth_right_z","mouth_right_visibility",
                                        "left_shoulder_x","left_shoulder_y","left_shoulder_z","left_shoulder_visibility",
                                        "right_shoulder_x","right_shoulder_y","right_shoulder_z","right_shoulder_visibility",
                                        "left_elbow_x","left_elbow_y","left_elbow_z","left_elbow_visibility",
                                        "right_elbow_x","right_elbow_y","right_elbow_z","right_elbow_visibility",
                                        "left_wrist_x","left_wrist_y","left_wrist_z","left_wrist_visibility",
                                        "right_wrist_x","right_wrist_y","right_wrist_z","right_wrist_visibility",
                                        "left_pinky_x","left_pinky_y","left_pinky_z","left_pinky_visibility",
                                        "right_pinky_x","right_pinky_y","right_pinky_z","right_pinky_visibility",
                                        "left_index_x","left_index_y","left_index_z","left_index_visibility",
                                        "right_index_x","right_index_y","right_index_z","right_index_visibility",
                                        "left_thumb_x","left_thumb_y","left_thumb_z","left_thumb_visibility",
                                        "right_thumb_x","right_thumb_y","right_thumb_z","right_thumb_visibility",
                                        "left_hip_x","left_hip_y","left_hip_z","left_hip_visibility",
                                        "right_hip_x","right_hip_y","right_hip_z","right_hip_visibility",
                                        "left_knee_x","left_knee_y","left_knee_z","left_knee_visibility",
                                        "right_knee_x","right_knee_y","right_knee_z","right_knee_visibility",
                                        "left_ankle_x","left_ankle_y","left_ankle_z","left_ankle_visibility",
                                        "right_ankle_x","right_ankle_y","right_ankle_z","right_ankle_visibility",
                                        "left_heel_x","left_heel_y","left_heel_z","left_heel_visibility",
                                        "right_heel_x","right_heel_y","right_heel_z","right_heel_visibility",
                                        "left_foot_index_x","left_foot_index_y","left_foot_index_z","left_foot_index_visibility",
                                        "right_foot_index_x","right_foot_index_y","right_foot_index_z","right_foot_index_visibility", 
                                        "pose","right_shoulder_angles","right_hip_angles","right_ankle_angles"]#,"dev"]
            
            first_column = df_puntos_final.pop('pose')
            
            df_puntos_final.insert(0, 'pose', first_column)
            
            # CAMBIAR DE ORDEN, LA POSE DEBE SER LA PRIMERA COLUMNA
            df_puntos_final.to_csv("./nuevas imagenes/front_plank/Front_Plank_puntos_trainer"+str(experto)+".csv", index=False)
            df_puntos_final = pd.DataFrame()
            experto = experto+1
            c = 0
        cv2.imshow("Image", image)
        cv2.destroyAllWindows()

Pose landmarks:  landmark {
  x: 0.0790972113609314
  y: 0.3902772068977356
  z: -0.07649204879999161
  visibility: 0.9998112320899963
}
landmark {
  x: 0.06849420070648193
  y: 0.3422839641571045
  z: -0.10214976221323013
  visibility: 0.9998376369476318
}
landmark {
  x: 0.06960129737854004
  y: 0.33559924364089966
  z: -0.10219380259513855
  visibility: 0.9998383522033691
}
landmark {
  x: 0.0707533061504364
  y: 0.3278542160987854
  z: -0.10232318192720413
  visibility: 0.9998700618743896
}
landmark {
  x: 0.06757521629333496
  y: 0.34410974383354187
  z: -0.05937591567635536
  visibility: 0.9996953010559082
}
landmark {
  x: 0.06794324517250061
  y: 0.3392094373703003
  z: -0.059383902698755264
  visibility: 0.9996308088302612
}
landmark {
  x: 0.0682879090309143
  y: 0.33341166377067566
  z: -0.05924837663769722
  visibility: 0.9997408986091614
}
landmark {
  x: 0.09122353792190552
  y: 0.28987208008766174
  z: -0.15121357142925262
  visibility: 0.9997164607048035
}
landmark {
  

Pose landmarks:  landmark {
  x: 0.09541525691747665
  y: 0.33646124601364136
  z: -0.07666957378387451
  visibility: 0.9997616410255432
}
landmark {
  x: 0.10324078798294067
  y: 0.2881157398223877
  z: -0.10128013044595718
  visibility: 0.9998437166213989
}
landmark {
  x: 0.10655626654624939
  y: 0.28545400500297546
  z: -0.10134419798851013
  visibility: 0.999850869178772
}
landmark {
  x: 0.11000138521194458
  y: 0.2823346257209778
  z: -0.10145249962806702
  visibility: 0.999866247177124
}
landmark {
  x: 0.10094448924064636
  y: 0.286447674036026
  z: -0.05057084187865257
  visibility: 0.9995556473731995
}
landmark {
  x: 0.10274955630302429
  y: 0.28313586115837097
  z: -0.05053547024726868
  visibility: 0.99951171875
}
landmark {
  x: 0.10498586297035217
  y: 0.27888286113739014
  z: -0.050328269600868225
  visibility: 0.9996598958969116
}
landmark {
  x: 0.13579359650611877
  y: 0.2806611657142639
  z: -0.15290996432304382
  visibility: 0.999687671661377
}
landmark {
  x: 0.1

Pose landmarks:  landmark {
  x: 0.16628718376159668
  y: 0.360320508480072
  z: 0.029676033183932304
  visibility: 0.9999196529388428
}
landmark {
  x: 0.16893060505390167
  y: 0.3112016022205353
  z: 0.0037512716371566057
  visibility: 0.9999147653579712
}
landmark {
  x: 0.17114165425300598
  y: 0.30826905369758606
  z: 0.003683472750708461
  visibility: 0.999922513961792
}
landmark {
  x: 0.17358922958374023
  y: 0.30510881543159485
  z: 0.0036127015482634306
  visibility: 0.9999357461929321
}
landmark {
  x: 0.1679355502128601
  y: 0.3099200129508972
  z: 0.04736828804016113
  visibility: 0.9998677968978882
}
landmark {
  x: 0.16932237148284912
  y: 0.30625295639038086
  z: 0.04734084755182266
  visibility: 0.9998599290847778
}
landmark {
  x: 0.17116114497184753
  y: 0.30220189690589905
  z: 0.04741944000124931
  visibility: 0.9999136924743652
}
landmark {
  x: 0.19583255052566528
  y: 0.29900631308555603
  z: -0.05478275939822197
  visibility: 0.9998495578765869
}
landmark {
  x

### Matriz de costos de imágenes

In [179]:
df_experto1 = pd.read_csv("./nuevas imagenes/front_plank/Front_Plank_puntos_trainer2.csv")
del df_experto1['pose']
del df_experto1['right_shoulder_angles']
del df_experto1['right_hip_angles']
del df_experto1['right_ankle_angles']
#del df_experto1['dev']
df_experto1

,nose_x,nose_y,nose_z,nose_visibility,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_inner_visibility,left_eye_x,left_eye_y,...,right_heel_z,right_heel_visibility,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_visibility,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_visibility
0,0.095415,0.336461,-0.076670,0.999762,0.103241,0.288116,-0.10128,0.999844,0.106556,0.285454,...,0.343658,0.878652,0.954717,0.896260,-0.281062,0.994378,0.890288,0.725863,0.300832,0.950321
1,0.082260,0.476098,-0.014761,0.999976,0.068385,0.430953,-0.05117,0.999976,0.069540,0.424081,...,0.361719,0.728648,0.887634,0.865795,-0.385783,0.967853,0.854510,0.755876,0.319216,0.793949
2,0.095415,0.336461,-0.076670,0.999762,0.103241,0.288116,-0.10128,0.999844,0.106556,0.285454,...,0.343658,0.878652,0.954717,0.896260,-0.281062,0.994378,0.890288,0.725863,0.300832,0.950321


In [180]:
df_experto2 = pd.read_csv("./nuevas imagenes/front_plank/Front_Plank_puntos_trainer3.csv")
del df_experto2['pose']
del df_experto2['right_shoulder_angles']
del df_experto2['right_hip_angles']
del df_experto2['right_ankle_angles']
#del df_experto2['dev']
df_experto2

,nose_x,nose_y,nose_z,nose_visibility,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_inner_visibility,left_eye_x,left_eye_y,...,right_heel_z,right_heel_visibility,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_visibility,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_visibility
0,0.166287,0.360321,0.029676,0.999920,0.168931,0.311202,0.003751,0.999915,0.171142,0.308269,...,0.231227,0.787773,0.936613,0.831693,-0.232065,0.989632,0.888517,0.810472,0.178862,0.900766
1,0.144475,0.419172,0.047548,0.999908,0.138811,0.372148,0.015204,0.999912,0.139734,0.368680,...,0.109613,0.699008,0.877963,0.803217,-0.223428,0.938084,0.856484,0.771387,0.052068,0.767487
2,0.166287,0.360321,0.029676,0.999920,0.168931,0.311202,0.003751,0.999915,0.171142,0.308269,...,0.231227,0.787773,0.936613,0.831693,-0.232065,0.989632,0.888517,0.810472,0.178862,0.900766


In [181]:
c=0

matriz_costos = []

inicio=0
fin = 1

while inicio < len(df_experto1):
    
    ej_usuario = []
    
    for i in df_experto2.columns:
        ej_usuario.append(df_experto2[i][inicio])
    
    resultados_costos = calcular_costos(ej_usuario,df_experto1,inicio,fin)
    matriz_costos.append(resultados_costos[0][1])
    inicio = inicio+1
matriz_costos = np.reshape(matriz_costos, (3, 1)) #(5,1) para 5 poses; (3,1) para 3 poses

In [182]:
matriz_costos

array([[6.75324539],
       [9.24466094],
       [6.75324539]])

In [183]:
df_matriz_costos = pd.DataFrame(matriz_costos)
df_matriz_costos['Pose'] = [num for num in range(1,len(df_matriz_costos)+1)]
df_matriz_costos.columns = ["Costo_alineamiento","Pose"]
df_matriz_costos = df_matriz_costos[["Pose","Costo_alineamiento"]]
#df_matriz_costos = df_matriz_costos.head()
#df_matriz_costos['Desviacion_estandar'] = 0
df_matriz_costos

,Pose,Costo_alineamiento
0,1,6.753245
1,2,9.244661
2,3,6.753245


In [184]:
df_matriz_costos.to_csv("./nuevas imagenes/front_plank/Costos_Front_Plank3.csv", index = False)

In [185]:
df1 = pd.read_csv("./nuevas imagenes/front_plank/Costos_Front_Plank1.csv")
#df1 = df1.head()
df2 = pd.read_csv("./nuevas imagenes/front_plank/Costos_Front_Plank2.csv")
#df2 = df2.head()
df3 = pd.read_csv("./nuevas imagenes/front_plank/Costos_Front_Plank3.csv")
#df3 = df3.head()

In [186]:
df_matriz_costos = pd.concat([df1,df2,df3])
df_matriz_costos

,Pose,Costo_alineamiento
0,1,4.561656
1,2,6.914216
2,3,4.561656
0,1,6.492741
1,2,7.753726
2,3,6.492741
0,1,6.753245
1,2,9.244661
2,3,6.753245


In [187]:
df = df_matriz_costos.copy()

In [188]:
df_final = pd.DataFrame()
c=0

poses = []
costos = []
stds = []
for i in df['Pose'].unique():
    
    df_ = df[df['Pose'] == i]
    df_ = df_.reset_index()
    del df_['index']
    #print(df_)
    #df_ = df_.head()
    #df_

    if len(df_) > 1:
        print("Para el segundo ", str(i), " tiene ", len(df_), " registros")
        
        df0 = pd.DataFrame()
        
        #for i in df_.columns:
            
        valores = df_["Costo_alineamiento"].unique()
        valores = list(valores)
        std = np.std(valores)
        valores = np.median(valores)
        print("Costo: ", valores)
        print("Std: ", str(std))
        
        poses.append(i)
        costos.append(valores)
        stds.append(std)
    c=c+1
df_final['Pose'] = poses
df_final['Costo_alineamiento'] = costos
df_final['Desviacion_estandar'] = stds
df_final

Para el segundo  1  tiene  3  registros
Costo:  6.4927409978117785
Std:  0.9775260122463323
Para el segundo  2  tiene  3  registros
Costo:  7.753725880756974
Std:  0.9637103402427085
Para el segundo  3  tiene  3  registros
Costo:  6.4927409978117785
Std:  0.9775260122463323


,Pose,Costo_alineamiento,Desviacion_estandar
0,1,6.492741,0.977526
1,2,7.753726,0.963710
2,3,6.492741,0.977526


In [189]:
df_final.to_csv("./nuevas imagenes/front_plank/Costos_Front_Plank_promedio.csv", index=False)

### Matriz de angulos

In [162]:
df1 = pd.read_csv("./nuevas imagenes/front_plank/Front_Plank_puntos_trainer1.csv")
df1 = df1[["pose","right_shoulder_angles","right_hip_angles","right_ankle_angles"]]
df2 = pd.read_csv("./nuevas imagenes/front_plank/Front_Plank_puntos_trainer2.csv")
df2 = df2[["pose","right_shoulder_angles","right_hip_angles","right_ankle_angles"]]
df3 = pd.read_csv("./nuevas imagenes/front_plank/Front_Plank_puntos_trainer3.csv")
df3 = df3[["pose","right_shoulder_angles","right_hip_angles","right_ankle_angles"]]

In [163]:
df_matriz_angulos = pd.concat([df1,df2,df3])
df_matriz_angulos

,pose,right_shoulder_angles,right_hip_angles,right_ankle_angles
0,1,53.032877,156.005500,150.715495
1,2,70.427818,175.757986,88.575884
2,3,53.032877,156.005500,150.715495
0,1,56.122159,157.377140,152.129574
1,2,91.217633,152.740814,98.787756
2,3,56.122159,157.377140,152.129574
0,1,70.042797,158.814171,120.152146
1,2,88.411701,177.509931,119.777776
2,3,70.042797,158.814171,120.152146


In [164]:
df = df_matriz_angulos.copy()

In [165]:
df_final = pd.DataFrame()
c=0

poses = []
angulos = []
stds = []
parte = []
partes = ["right_shoulder_angles","right_hip_angles","right_ankle_angles"]
for j in df['pose'].unique():
    for i in partes:

        df_ = df[["pose",i]][df['pose'] == j]
        df_ = df_.reset_index()
        del df_['index']
        #print(df_)
        #df_ = df_.head()
        #df_

        if len(df_) > 1:
            print("Para la pose ", str(j), " tiene ", len(df_), " registros")

            df0 = pd.DataFrame()

            #for i in df_.columns:

            valores = df_[i].unique()
            valores = list(valores)
            std = np.std(valores)
            valores = np.median(valores)
            print("Angulo: ", valores)
            print("Std: ", str(std))

            poses.append(j)
            angulos.append(valores)
            stds.append(std)
            parte.append(i)
        c=c+1
df_final['pose'] = poses
df_final['Angulo'] = angulos
df_final['Desviacion_estandar'] = stds
df_final['Articulacion'] = parte
df_final

Para la pose  1  tiene  3  registros
Angulo:  56.12215935203271
Std:  7.398687671633633
Para la pose  1  tiene  3  registros
Angulo:  157.37713991572747
Std:  1.1467386148985588
Para la pose  1  tiene  3  registros
Angulo:  150.71549478914085
Std:  14.752302269645408
Para la pose  2  tiene  3  registros
Angulo:  88.41170140631407
Std:  9.210559909645786
Para la pose  2  tiene  3  registros
Angulo:  175.75798631961402
Std:  11.286022393991257
Para la pose  2  tiene  3  registros
Angulo:  98.78775628756284
Std:  12.988975112237423
Para la pose  3  tiene  3  registros
Angulo:  56.12215935203271
Std:  7.398687671633633
Para la pose  3  tiene  3  registros
Angulo:  157.37713991572747
Std:  1.1467386148985588
Para la pose  3  tiene  3  registros
Angulo:  150.71549478914085
Std:  14.752302269645408


,pose,Angulo,Desviacion_estandar,Articulacion
0,1,56.122159,7.398688,right_shoulder_angles
1,1,157.377140,1.146739,right_hip_angles
2,1,150.715495,14.752302,right_ankle_angles
3,2,88.411701,9.210560,right_shoulder_angles
4,2,175.757986,11.286022,right_hip_angles
5,2,98.787756,12.988975,right_ankle_angles
6,3,56.122159,7.398688,right_shoulder_angles
7,3,157.377140,1.146739,right_hip_angles
8,3,150.715495,14.752302,right_ankle_angles


In [166]:
df_final.to_csv("./nuevas imagenes/front_plank/Angulos_Front_Plank_promedio.csv", index=False)